In [1]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import tensorflow as tf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
pip install git+https://github.com/MihaMarkic/tflearn.git@fix/is_sequence_missing

  Cloning https://github.com/MihaMarkic/tflearn.git (to revision fix/is_sequence_missing) to /tmp/pip-req-build-ptcm0ukl
  Running command git clone --filter=blob:none --quiet https://github.com/MihaMarkic/tflearn.git /tmp/pip-req-build-ptcm0ukl
  Running command git checkout -b fix/is_sequence_missing --track origin/fix/is_sequence_missing
  Switched to a new branch 'fix/is_sequence_missing'
  Branch 'fix/is_sequence_missing' set up to track remote branch 'fix/is_sequence_missing' from 'origin'.
  Resolved https://github.com/MihaMarkic/tflearn.git to commit 6472b8588e758ff4a33a2764d4ee638bbd0e42f0
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=130659 sha256=3da21c7f3412b31fe4417df6cd903cb6c852ad5b21e59a4971745d9e2bda480b
  Stored in directory: /tmp/pip-ephem-wheel-cache-x3s_e23a/wheels/4d/8c/0c/2159783f8dfd53bdaf4e59cd3990607ac8d5d912ae6de88296
Successfully built tflearn


In [3]:
import numpy
import tensorflow as tf
import tflearn
import random
import json
import pickle
from time import sleep

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split


with open("intents2.json") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
    for intent in data ["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)


        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)
    X_train, X_test, y_train, y_test = train_test_split(training, output, test_size=0.2)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(X_train[0])])
net = tflearn.fully_connected(net, 20)
net = tflearn.fully_connected(net, 20)
net = tflearn.fully_connected(net, 20)
net = tflearn.fully_connected(net, len(y_train[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(X_train, y_train, n_epoch=200, batch_size=8, show_metric=True)
model.save("model.tflearn")
y_train_pred = model.predict(X_train)
f1_train = f1_score(y_train.argmax(axis=1), y_train_pred.argmax(axis=1), average='weighted')
print(f"F1 Score (Training): {f1_train:.4f}")

test_loss = model.evaluate(X_test, y_test)[0]
print(f"Test Loss: {test_loss:.4f}")
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

def chat():
    print("Hi, How can i help you ?")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        if results[results_index] > 0.8:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            sleep(3)
            Bot = random.choice(responses)
            print(Bot)
        else:
            print("I don't understand!")
chat()

Training Step: 6399  | total loss: 0.44847 | time: 0.199s
| Adam | epoch: 200 | loss: 0.44847 - acc: 0.8707 -- iter: 248/256
Training Step: 6400  | total loss: 0.43654 | time: 0.207s
| Adam | epoch: 200 | loss: 0.43654 - acc: 0.8953 -- iter: 256/256
--
F1 Score (Training): 0.9422
Test Loss: 0.1538
Hi, How can i help you ?
You: how are you
Hello! How may I assist you?
You: what is the time 
Right now, it is [current time] [current time zone].
You: x
I don't understand!
You: how are you
Hello! How can I assist you?
You: boring
I don't understand!
You: what is the time 
Right now, it is [current time] [current time zone].
